    Ben Christensen
    Math 323
    March 10, 2018

Code a simplex solver and use it to solve an optimization problem.

In [1]:
import numpy as np

In [2]:
# Problems 1-6
class SimplexSolver(object):
    """Class for solving the standard linear optimization problem

                        maximize        c^Tx
                        subject to      Ax <= b
                                         x >= 0
    via the Simplex algorithm.

    Attributes:
        self.m, self.n (int, int): Shape of matrix A
        self.c ((n,) ndarray): c
        self.A ((m,n) ndarray): A
        self.b ((m,) ndarray): b
        index_list((m+n, 1) ndarray): Indexes for locating
                                      basic and non-basic variables

    """
    def __init__(self, c, A, b):
        """Check for feasibility and initialize the tableau.

        Parameters:
            c ((n,) ndarray): The coefficients of the objective function.
            A ((m,n) ndarray): The constraint coefficients matrix.
            b ((m,) ndarray): The constraint vector.

        Raises:
            ValueError: if the given system is infeasible at the origin.
        """
        for entry in b:
            if entry < 0:
                raise ValueError("Problem is not feasible at x=0")
        self.m, self.n = A.shape
        self.c = c
        self.A = A
        self.b = b
        #Create and store index list
        self.index_list = np.concatenate((np.arange(self.n, self.m+self.n, 1), np.arange(0, self.n, 1)))


    def tablate(self):
        """Initialize Tableau

        Attributes:
            T ((m+1, m+n+2) ndarray): Matrix represenation of tableau
        """

        self.T = np.hstack((np.insert(self.b, 0, 0).reshape((-1, 1)),
                            np.vstack((-np.append(self.c, np.zeros(self.m)),
                                        np.hstack((self.A, np.eye(self.m))))),
                            np.insert(np.zeros(self.m), 0, 1).reshape((-1, 1))))

    def find_indices(self):
        """Find pivot row and pivot column

        Returns:
            (int): Pivot Row
            (int): Pivot Column
        """
        #Find pivot column
        T = self.T
        for j in range(1, self.m+self.n+2):
            if T[0, j] < 0:
                break
        minimum = -1
        pivot_column = j
        #Find pivot row in pivot column
        for i in range(1, self.m+1):
            if T[i, j] > 0:
                temp = T[i,0] / T[i,j]
                if temp < minimum or minimum == -1:
                    minimum = temp
                    pivot_row = i
                #Apply bland's rule:
                elif temp == minimum and self.index_list[i] < self.index_list[pivot_row]:
                    pivot_row = i
        #If no pivot row is feasible the problem is unbounded
        if minimum == -1:
            raise ValueError("Problem is unbounded")

        return pivot_row, pivot_column

    def pivot(self):
        """Perform 1 pivot and update self.index_list accordingly"""
        #find.indices() checks for unboundedness and raises an error if appropriate
        i,j = self.find_indices()
        #Swap indices for old and new basic variable
        self.index_list[j - 1 + self.m], self.index_list[i-1] = self.index_list[i-1], self.index_list[j - 1 + self.m]
        self.T[i] = self.T[i] / self.T[i,j]
        #Row reduce
        for k in range(self.m+1):
            if k != i:
                temp = -self.T[k,j] / self.T[i,j]
                self.T[k] = self.T[k] + temp*self.T[i]

    def solve(self):
        """Solve the linear optimization problem.

        Returns:
            (float) The maximum value of the objective function.
            (dict): The basic variables and their values.
            (dict): The nonbasic variables and their values.
        """
        #Initialize Tableau
        self.tablate()
        #Pivot until all elements of the top row are positive
        while (self.T[0] < 0).any() == True:
            self.pivot()

        return self.T[0,0], dict(sorted(zip(self.index_list[0:self.m], self.T[1:,0]))), dict(sorted(zip(self.index_list[self.m:], np.zeros(self.n))))


# Problem 7
def prob7(filename='/Users/benchristensen/Desktop/ACME Python Labs/Volume2-Student-Materials/Simplex/productMix.npz'):
    """Solve the product mix problem for the data in 'productMix.npz'.

    Parameters:
        filename (str): the path to the data file.

    Returns:
        The minimizer of the problem (as an array).
    """
    #Put in standard form for simplex
    data = np.load(filename)
    A, c, b, d = data['A'], data['p'], data['m'], data['d']
    m,n = A.shape
    A = np.vstack((A, np.eye(n)))
    b = np.concatenate((b, d))
    #Apply simplex solver
    solver = SimplexSolver(c, A, b)
    solution = solver.solve()[1]
    #Format solution for easier reading
    products = []
    amounts = []
    for key in solution.keys():
        if key < 4:
            products.append(key)
            amounts.append(solution[key])
    #print(dict(zip(products, amounts)))
    return np.array(amounts)


In [3]:
#Testing the class
print("Class test:")
c = np.array([3., 2])
b = np.array([2., 5, 7])
A = np.array([[1., -1], [3, 1], [4, 3]])
solver = SimplexSolver(c, A, b)
sol = solver.solve()
print(sol)
#Testing Problem 7
print("Problem 7:")
print(prob7())

Class test:
(5.2000000000000002, {0: 1.6000000000000001, 1: 0.19999999999999982, 2: 0.59999999999999964}, {3: 0.0, 4: 0.0})
Problem 7:
[ 10.           6.19298246  12.           1.78947368]
